In [1]:
import boto3
import pandas as pd
import numpy as np
import sys
from io import StringIO
import surprise
from surprise import accuracy
from sklearn.model_selection import train_test_split
from surprise import SVD, NMF, KNNBaseline
import pickle


In [2]:
sys.path.insert(1, '../src')
from Collaborative_recommender import CollaborativeRecommender
from Content_recommender import ContentRecommender
from NLP import nlp_process

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
%reload_ext autoreload

In [5]:
s3 = boto3.client('s3')

In [6]:
ls ../../data

recom_colab.pkl


In [8]:
with open('../../data/recom_colab.pkl', 'rb') as f:
    recom_colab = pickle.load(f)

In [9]:
UI = "AZZG39P6RDCHL"
user_preference = recom_colab.get_top_n(UI,n=1)

In [10]:
user_preference

,predicted_rating
iid,
B000P3WQ86,5.0


In [6]:
df_all_15 = pd.read_csv("s3://recommender-system-amazon/df_all_15_text.csv")

### Prepare required dataframe for the hybrid recommender 

In [7]:
df_items_text = df_all_15.groupby(['asin', 'title','title_nlp','image_x'])['item_text'].apply(lambda x: ' '.join(x)).reset_index()

In [8]:
df_collab = df_all_15[['reviewerID',"asin","overall"]]

In [33]:
df_content = pd.read_csv("s3://recommender-system-amazon/df_content.csv", index_col=[0])

### Collaborative filter

In [17]:
reader = surprise.Reader(rating_scale=(1,5))
recom_colab = CollaborativeRecommender(df_collab,reader,NMF)
param_grid = {'n_factors': [8,15,23,30,37,42], 'n_epochs': [5, 10]}
recom_colab.grid(param_grid,cv=3)
recom_colab.fit()

In [21]:
df_pred, acc = recom_colab.pred()
UM = recom_colab.utility_matrix()

RMSE: 0.6924


In [38]:
UI = "AZZG39P6RDCHL"
user_preference = recom_colab.get_top_n(UI,n=1)

### Content based filter

In [36]:
recom_content = ContentRecommender()
similarity_df = recom_content.fit(df_content, items = df_content.index)

In [44]:
item = user_preference.index
recom_content.get_recommendations(item='B01HCU3NTU')

array(['B01BBTS3JI', 'B007UT3OFC', 'B00C2NQ6XS', 'B00PRVLMK4',
       'B00457SRBI'], dtype=object)